In [ ]:
pip install mlxtend

In [ ]:
pip install mlxtend --upgrade

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#dataset source
#https://www.kaggle.com/heeraldedhia/groceries-dataset
df = pd.read_csv('https://raw.githubusercontent.com/ganjar87/data_science_practice/main/Groceries_dataset.csv', delimiter=',')
df.columns


Index(['Member_number', 'Date', 'itemDescription'], dtype='object')

## dataset preparation

In [ ]:
df_group = df.groupby(['Member_number','Date'])['itemDescription'].agg(lambda x: ','.join(x.dropna())).reset_index()
df_group.sort_values(by='Date', ascending=True, inplace=True,ignore_index=True)
df_group.head(10)

,Member_number,Date,itemDescription
0,4260,01-01-2014,"soda,brown bread"
1,1659,01-01-2014,"specialty chocolate,frozen vegetables"
2,3797,01-01-2014,"waffles,whole milk"
3,1440,01-01-2014,"other vegetables,yogurt"
4,3942,01-01-2014,"other vegetables,yogurt,Instant food products"
5,2351,01-01-2014,"cleaner,shopping bags"
6,2943,01-01-2014,"whole milk,flower (seeds)"
7,1381,01-01-2014,"curd,soda"
8,4942,01-01-2014,"butter,frozen vegetables"
9,2610,01-01-2014,"hamburger meat,bottled beer,domestic eggs"


In [ ]:
dataset = []
for i in range(len(df_group['itemDescription'])):
  new_val = df_group['itemDescription'].iloc[i].split(',')
  dataset.append(new_val)

In [ ]:
len(df_group['itemDescription'])

14963

In [ ]:
df_group['itemDescription'].iloc[0].split(',')

['soda', 'brown bread']

## Frequent Pattern

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth


te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

df

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,...,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
4,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14958,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False,False,False,False,Fa

In [ ]:
frequent_itemsets = fpgrowth(df, min_support=0.005, use_colnames=True)
### alternatively:
#frequent_itemsets = apriori(df, min_support=0.005, use_colnames=True)
#frequent_itemsets = fpmax.fpmax(df, min_support=0.005, use_colnames=True)

frequent_itemsets

,support,itemsets
0,0.097106,(soda)
1,0.037626,(brown bread)
2,0.028002,(frozen vegetables)
3,0.015973,(specialty chocolate)
4,0.157923,(whole milk)
...,...,...
745,0.001403,"(other vegetables, hygiene articles)"
746,0.001337,"(rolls/buns, hygiene articles)"
747,0.001069,"(soda, hygiene articles)"
748,0.001738,"(whole milk, hygiene articles)"


In [ ]:
frequent_itemsets.sort_values(by=['support'], axis=0, inplace=False, ascending = False)

,support,itemsets
4,0.157923,(whole milk)
6,0.122101,(other vegetables)
52,0.110005,(rolls/buns)
0,0.097106,(soda)
7,0.085879,(yogurt)
...,...,...
620,0.001002,"(root vegetables, misc. beverages)"
615,0.001002,"(rolls/buns, soft cheese)"
595,0.001002,"(chicken, root vegetables)"
594,0.001002,"(chicken, bottled beer)"


In [ ]:
#get better data structure
frequent_itemsets['num_item']=frequent_itemsets.apply(lambda x:len(x['itemsets']), axis=1)
frequent_itemsets['item_src']=frequent_itemsets.apply(lambda x:", ".join(x['itemsets']), axis=1)
frequent_itemsets.head()

,support,itemsets,num_item,item_src
0,0.097106,(soda),1,soda
1,0.037626,(brown bread),1,brown bread
2,0.028002,(frozen vegetables),1,frozen vegetables
3,0.015973,(specialty chocolate),1,specialty chocolate
4,0.157923,(whole milk),1,whole milk


In [ ]:
tet = frequent_itemsets[frequent_itemsets['num_item']>2]
tet.head()

,support,itemsets,num_item,item_src
152,0.001136,"(soda, whole milk, other vegetables)",3,"soda, whole milk, other vegetables"
153,0.001002,"(rolls/buns, soda, whole milk)",3,"rolls/buns, soda, whole milk"
154,0.001136,"(rolls/buns, soda, other vegetables)",3,"rolls/buns, soda, other vegetables"
204,0.001136,"(other vegetables, whole milk, yogurt)",3,"other vegetables, whole milk, yogurt"
205,0.001337,"(rolls/buns, whole milk, yogurt)",3,"rolls/buns, whole milk, yogurt"


In [ ]:
frequent_itemsets[frequent_itemsets['item_src'].str.contains('yogurt')].sort_values(by=['support'], axis=0, inplace=False, ascending = False)